In [ ]:

import os, sys, numpy as np, pandas as pd, torch
sys.path.append("..")
from src.config import ModelParams, TrainConfig
from src.deqn import PolicyNetwork, Trainer
from src.steady_states import solve_flexprice_sss, export_rbar_tensor, solve_taylor_sss
from src.io_utils import ensure_dir, save_torch, save_csv, save_json, pack_config, save_selected_run


In [ ]:
policy = "mod_taylor"
params = ModelParams(device="cpu", dtype=torch.float32)

# --- reproducible run directory (unique; safe for parallel runs) ---
from src.io_utils import make_run_dir, save_run_metadata, pack_config
run_dir = make_run_dir("../artifacts", policy=policy, tag="cpu16gb_plan", seed=123)

# Strict training plan for a 16GB RAM CPU box:
#   Stage A: train with gh_n=3 (main training)
#   Stage B: short fine-tune with gh_n=5 (more accurate quadrature), with early stopping

cfgA = TrainConfig(
    seed=123,
    steps=20000,
    batch_size=256,          # keep as requested
    lr=1e-05,
    grad_clip=1.0,
    gh_n_train=3,
    replay_buffer_size=250_000,
    replay_refresh_every=120,
    replay_refresh_steps=80,
    replay_init_fill_steps=400,
    val_size=20000,
    val_every=500,
    reduce_lr_on_plateau=True,
    plateau_patience=3000,
    lr_reduce_factor=0.5,
    min_lr=2e-06,
    artifacts_root="../artifacts",
    run_dir=run_dir,
    save_best=True,
    enable_timers=True,
    log_every=50,
    profile=False,
    cpu_num_threads=None,
    cpu_num_interop_threads=None,
    matmul_precision="high",
)

cfgB = TrainConfig(
    seed=123,
    steps=2000,
    batch_size=64,           # gh_n=5 is expensive: keep batch small
    lr=3e-06,
    grad_clip=1.0,
    gh_n_train=5,
    replay_buffer_size=250_000,
    replay_refresh_every=200,
    replay_refresh_steps=60,
    replay_init_fill_steps=0,
    val_size=20000,
    val_every=250,
    early_stopping=True,
    patience=1500,
    min_delta=1e-5,
    artifacts_root="../artifacts",
    run_dir=run_dir,
    save_best=True,
    enable_timers=True,
    log_every=50,
    profile=False,
    cpu_num_threads=None,
    cpu_num_interop_threads=None,
    matmul_precision="high",
)


In [ ]:

flex = solve_flexprice_sss(params)
rbar = export_rbar_tensor(params, flex)


# For modified Taylor, theoretical long-run inflation is 0 by construction (intercept tracks regime natural rate).
# We still compute standard Taylor SSS via eq.(25) for comparison.
taylor_sss = solve_taylor_sss(params, flex)
print('Standard Taylor SSS (for comparison):', taylor_sss.by_regime)


In [ ]:
net = PolicyNetwork(d_in=5, d_out=8, hidden=cfgA.hidden_layers, activation=cfgA.activation)

# Phase 1
trainer = Trainer(params=params, cfg=cfgA, policy=policy, net=net, rbar_by_regime=rbar if policy=="mod_taylor" else None)
losses_1 = trainer.train(B=1024, commitment_sss=commitment_sss if policy=="commitment" else None, burn_in=500, fine_tune_float64=False)

# Cleanup between phases (helps on Windows/8GB RAM)
import gc
_ = gc.collect()
print('gc.collect() done (between phases)')

# Phase 2 (warm start)
trainer = Trainer(params=params, cfg=cfgB, policy=policy, net=net, rbar_by_regime=rbar if policy=="mod_taylor" else None)
losses_2 = trainer.train(B=1024, commitment_sss=commitment_sss if policy=="commitment" else None, burn_in=500, fine_tune_float64=False)

losses = losses_1 + losses_2
print(losses[:5], losses[-5:])


In [ ]:
from src.deqn import simulate_paths
x0 = trainer.simulate_initial_state(256)
sim = simulate_paths(params, policy, trainer.net, T=3000, burn_in=600, x0=x0, rbar_by_regime=rbar)
for s in [0,1]:
    mask = (sim['s'] == s)
    print(f'Regime {s}: pi_sim={sim["pi"][mask].mean():.6f} (should be ~0 under mod_taylor)')


In [ ]:
# --- Save artifacts (final + pointers) ---
# We save final weights/logs to run_dir and also keep a pointer in selected_runs.json

# prefer Stage B config if it exists, else Stage A
_cfg_final = globals().get("cfgB", globals().get("cfgA"))
save_run_metadata(run_dir, pack_config(params, _cfg_final, extra={"policy": policy}))

save_torch(os.path.join(run_dir, "weights_final.pt"), trainer.net.state_dict())
# best checkpoint is saved automatically during training as cfg.best_weights_name when val improves

df = pd.DataFrame({"step": np.arange(len(losses)), "loss": losses})
save_csv(os.path.join(run_dir, "train_log.csv"), df)

meta = pack_config(params, _cfg_final, extra={"policy": policy, "flex_sss": flex.by_regime})
save_json(os.path.join(run_dir, "meta.json"), meta)

pointer_path = save_selected_run(_cfg_final.artifacts_root, policy, run_dir)
print("Saved to", run_dir)
print("Selected run pointer:", pointer_path)


In [ ]:
# Final cleanup (helps when running multiple models sequentially)
import gc
try:
    del trainer
except NameError:
    pass
try:
    del net
except NameError:
    pass
_ = gc.collect()
print('final gc.collect() done')
